In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import lightgbm as lgb

import sklearn.preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold

In [ ]:
train = pd.read_csv('../input/lish-moa/train_features.csv')
train.head()

In [ ]:
train.shape

In [ ]:
test = pd.read_csv("../input/lish-moa/test_features.csv")
test.head()

In [ ]:
train['cp_type'] = (train['cp_type'] == 'trt_cp').astype('int')
train['cp_dose'] = (train['cp_dose'] == 'D1').astype('int')

In [ ]:
test['cp_type'] = (test['cp_type'] == 'trt_cp').astype('int')
test['cp_dose'] = (test['cp_dose'] == 'D1').astype('int')

In [ ]:
train.info()

In [ ]:
train_NS = pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
train_NS.head()

In [ ]:
target = pd.read_csv('../input/lish-moa/train_targets_scored.csv')
target.head()

In [ ]:
target.shape

In [ ]:
train_drug = pd.read_csv('../input/lish-moa/train_drug.csv')
sub = pd.read_csv("../input/lish-moa/sample_submission.csv")

In [ ]:
label_encoder = sklearn.preprocessing.LabelEncoder()

In [ ]:
mapped_drug_id = pd.Series(label_encoder.fit_transform(train_drug['drug_id']))
train_drug['drug_id'] = mapped_drug_id
train_drug.head()

In [ ]:
train.merge(train_drug)

In [ ]:
def run_lgbm(target_col: str):
    
    X_train = train.drop(["sig_id"], axis=1) # train без sig_id
    y_train = target[target_col]             # столбец из цикла for ниже
    X_test = test.drop(["sig_id"], axis=1)   # test без sig_id

    y_preds = []
    models = []
    oof_train = np.zeros((len(X_train),))   # [0. 0. 0. ... 0. 0. 0.] длиной 23814

    for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train)):
        X_tr = X_train.loc[train_index, :]
        X_val = X_train.loc[valid_index, :]
        y_tr = y_train[train_index]
        y_val = y_train[valid_index]

        lgb_train = lgb.Dataset(X_tr,
                                y_tr,
                                categorical_feature=categorical_cols)

        lgb_eval = lgb.Dataset(X_val,
                               y_val,
                               reference=lgb_train,
                               categorical_feature=categorical_cols)

        model = lgb.train(params,
                          lgb_train,
                          valid_sets=[lgb_train, lgb_eval],
                          verbose_eval=10,
                          num_boost_round=1000,
                          early_stopping_rounds=10)


        oof_train[valid_index] = model.predict(X_val,
                                               num_iteration=model.best_iteration)
        y_pred = model.predict(X_test,
                               num_iteration=model.best_iteration)

        y_preds.append(y_pred)
        models.append(model)

    return oof_train, sum(y_preds) / len(y_preds)

In [ ]:
# X_train = train.drop(["sig_id"], axis=1)
# oof_train = np.zeros((len(X_train),))
# print(oof_train)

In [ ]:
# from catboost import CatBoostClassifier
# cat = CatBoostClassifier(random_seed=17)

# def run_cat(target_col: str):
    
#     X_train = train.drop(["sig_id"], axis=1) # train без sig_id
#     y_train = target[target_col]             # столбец из цикла for ниже
    
#     y_preds = []
#     models = []
#     oof_train = np.zeros((len(X_train),))
    
#     X_train_df, X_valid, y_train_df, y_valid = train_test_split(train_df, y, 
#                                                                 test_size=.2, stratify=y_train, 
#                                                                 random_state=17)
    
#     for train_index, valid_index in cv.split(X_train_df, y_train_df):
#         X_tr = X_train_df.iloc[train_index, :]
#         X_val = X_train_df.iloc[valid_index, :]
#         y_tr = y_train_df.iloc[train_index]
#         y_val = y_train_df.iloc[valid_index]
        
#         model = cat.fit(X_tr, y_tr, cat_features=categorical_cols)
        
#         oof_train[valid_index] = model.predict(X_val,
#                                                 num_iteration=model.best_iteration)
        
        

In [ ]:
cv = KFold(n_splits=5, shuffle=True, random_state=0)

params = {
    'num_leaves': 25,
    'max_depth': 5,
    'objective': 'binary',
    'learning_rate': 0.03
}

categorical_cols = ['cp_type', 'cp_dose']
oof = target.copy()

In [ ]:
for target_col in target.columns:
    if target_col != "sig_id":
        _oof, _preds = run_lgbm(target_col)
        oof[target_col] = _oof
        sub[target_col] = _preds

In [ ]:
scores = []
for target_col in target.columns:
    if target_col != "sig_id":
        scores.append(log_loss(target[target_col], oof[target_col]))
print(np.mean(scores))

In [ ]:
sub.to_csv('submission.csv', index=False)